# Titanic Survival Rate Predictions

### Importing packages

In [629]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


### Explorating Data

In [630]:
# Importing data csv as pandas df.
raw_train = pd.read_csv('train.csv')
raw_test = pd.read_csv('test.csv')

In [631]:
# General overview of the data
raw_train.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [632]:
# General information.
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [633]:
# Null values. Seems like we'll have some null values to handle in Age, Cabin, Fare and Embarked.
print(raw_train.isnull().sum())
print(raw_test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [634]:
raw_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [635]:
# There are no duplicates in out datasets.
dup_train = raw_train.duplicated().any()
dup_test = raw_test.duplicated().any()
print(dup_train)
print(dup_test)

False
False


In [636]:
raw_train["Age"].mode()[0]

24.0

### Data cleaning

In [637]:
# We'll replace NaN values for Age, Fare and Embarked. Cabin has too many NaN to be useful in our models.
# Where applicable, we'll always use the same values calculated for train in test. To avoid leakage.

raw_train["Age"] = raw_train["Age"].fillna(raw_train["Age"].median())
raw_test["Age"] = raw_test["Age"].fillna(raw_train["Age"].median())

raw_train["Embarked"] = raw_train["Embarked"].fillna(raw_train["Embarked"].mode()[0])
raw_test["Embarked"] = raw_test["Embarked"].fillna(raw_train["Embarked"].mode()[0])

raw_train["Fare"] = raw_train["Fare"].fillna(raw_train["Fare"].mean())
raw_test["Fare"] = raw_test["Fare"].fillna(raw_train["Fare"].mean())

In [638]:
print(raw_train.isnull().sum())
print(raw_test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [639]:
# Dictionary for numerical conversion of column sex
factors = {'male': 0, 'female': 1}

# Applying the same conversion to both df
raw_train['Sex'] = raw_train['Sex'].map(factors)
raw_test['Sex'] = raw_test['Sex'].map(factors)

In [640]:
raw_test["Embarked"].unique()

array(['Q', 'S', 'C'], dtype=object)

In [641]:
# Dictionary for numerical conversion of column Embarked
factors = {'S': 0, 'C': 1, 'Q': 2}

# Applying the same conversion to both df
raw_train['Embarked'] = raw_train['Embarked'].map(factors)
raw_test['Embarked'] = raw_test['Embarked'].map(factors)

In [642]:
raw_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


In [643]:
# Preparing files before starting to fit ML models to the data
passengerid_test = raw_test['PassengerId']
y_train = raw_train['Survived']
x_train = raw_train.drop(['PassengerId','Survived','Name', 'Ticket', 'Cabin'], axis=1)
x_test = raw_test.drop(['PassengerId','Name', 'Ticket', 'Cabin'], axis=1)

### Decision Tree

In [644]:
dtree = DecisionTreeClassifier(random_state=6)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=6)

In [645]:
predictions_tree = dtree.predict(x_train)

In [646]:
print(predictions_tree.mean())
print(y_train.mean())

0.3681257014590348
0.3838383838383838


### Random Forest

In [647]:
rfc = RandomForestClassifier(n_estimators=1000, random_state=6)
rfc.fit(x_train,y_train)

RandomForestClassifier(n_estimators=1000, random_state=6)

In [648]:
predictions_rfc = rfc.predict(x_test)

In [649]:
print(predictions_rfc.mean())
print(y_train.mean())

0.36363636363636365
0.3838383838383838


### Neural Network

In [650]:
x_train['Age'] = StandardScaler().fit_transform(x_train[['Age']])
x_test['Age'] = StandardScaler().fit_transform(x_test[['Age']])

x_train['Fare'] = StandardScaler().fit_transform(x_train[['Fare']])
x_test['Fare'] = StandardScaler().fit_transform(x_test[['Fare']])


In [651]:
columns_to_encode = ['Pclass','SibSp','Parch','Embarked']

train_df_encoded = pd.get_dummies(x_train, columns=columns_to_encode)
test_df_encoded = pd.get_dummies(x_test, columns=columns_to_encode)

In [652]:
train_df_encoded.head()

,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_0,Embarked_1,Embarked_2
0,0,-0.565736,-0.502445,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1,0.663861,0.786845,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,-0.258337,-0.488854,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,1,0.433312,0.420730,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0,0.433312,-0.486337,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [653]:
# Initialize the model
nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000, alpha=0.0001,
                     solver='adam', verbose=10,  random_state=21,tol=0.000000001)

# Fit the model to the training data
nn.fit(x_train, y_train)

Iteration 1, loss = 0.71518787
Iteration 2, loss = 0.68547307
Iteration 3, loss = 0.66272712
Iteration 4, loss = 0.64285876
Iteration 5, loss = 0.62545594
Iteration 6, loss = 0.60968258
Iteration 7, loss = 0.59603509
Iteration 8, loss = 0.58290934
Iteration 9, loss = 0.57145536
Iteration 10, loss = 0.56083268
Iteration 11, loss = 0.55068792
Iteration 12, loss = 0.54170329
Iteration 13, loss = 0.53163854
Iteration 14, loss = 0.52347509
Iteration 15, loss = 0.51556166
Iteration 16, loss = 0.50771419
Iteration 17, loss = 0.50017529
Iteration 18, loss = 0.49355782
Iteration 19, loss = 0.48697126
Iteration 20, loss = 0.48125549
Iteration 21, loss = 0.47555459
Iteration 22, loss = 0.47028133
Iteration 23, loss = 0.46554089
Iteration 24, loss = 0.46136652
Iteration 25, loss = 0.45705378
Iteration 26, loss = 0.45329041
Iteration 27, loss = 0.45004952
Iteration 28, loss = 0.44649878
Iteration 29, loss = 0.44379930
Iteration 30, loss = 0.44104774
Iteration 31, loss = 0.43841510
Iteration 32, los

Iteration 271, loss = 0.37383434
Iteration 272, loss = 0.37382358
Iteration 273, loss = 0.37369956
Iteration 274, loss = 0.37361888
Iteration 275, loss = 0.37339192
Iteration 276, loss = 0.37331199
Iteration 277, loss = 0.37316095
Iteration 278, loss = 0.37293798
Iteration 279, loss = 0.37301104
Iteration 280, loss = 0.37272511
Iteration 281, loss = 0.37287184
Iteration 282, loss = 0.37266780
Iteration 283, loss = 0.37240260
Iteration 284, loss = 0.37235388
Iteration 285, loss = 0.37275856
Iteration 286, loss = 0.37236653
Iteration 287, loss = 0.37188023
Iteration 288, loss = 0.37219919
Iteration 289, loss = 0.37180961
Iteration 290, loss = 0.37198453
Iteration 291, loss = 0.37190991
Iteration 292, loss = 0.37157896
Iteration 293, loss = 0.37136566
Iteration 294, loss = 0.37150712
Iteration 295, loss = 0.37144343
Iteration 296, loss = 0.37124346
Iteration 297, loss = 0.37127428
Iteration 298, loss = 0.37112430
Iteration 299, loss = 0.37102321
Iteration 300, loss = 0.37083649
Iteration 

Iteration 521, loss = 0.35208668
Iteration 522, loss = 0.35202494
Iteration 523, loss = 0.35205277
Iteration 524, loss = 0.35197563
Iteration 525, loss = 0.35218088
Iteration 526, loss = 0.35194746
Iteration 527, loss = 0.35174044
Iteration 528, loss = 0.35196718
Iteration 529, loss = 0.35158386
Iteration 530, loss = 0.35204324
Iteration 531, loss = 0.35185749
Iteration 532, loss = 0.35176543
Iteration 533, loss = 0.35157834
Iteration 534, loss = 0.35168552
Iteration 535, loss = 0.35186045
Iteration 536, loss = 0.35122024
Iteration 537, loss = 0.35104012
Iteration 538, loss = 0.35136941
Iteration 539, loss = 0.35149337
Iteration 540, loss = 0.35094666
Iteration 541, loss = 0.35097031
Iteration 542, loss = 0.35111434
Iteration 543, loss = 0.35097975
Iteration 544, loss = 0.35133027
Iteration 545, loss = 0.35054314
Iteration 546, loss = 0.35046747
Iteration 547, loss = 0.35111755
Iteration 548, loss = 0.35024638
Iteration 549, loss = 0.35060035
Iteration 550, loss = 0.35045021
Iteration 

Iteration 795, loss = 0.33680227
Iteration 796, loss = 0.33658660
Iteration 797, loss = 0.33675096
Iteration 798, loss = 0.33642605
Iteration 799, loss = 0.33669443
Iteration 800, loss = 0.33636171
Iteration 801, loss = 0.33645774
Iteration 802, loss = 0.33630474
Iteration 803, loss = 0.33637699
Iteration 804, loss = 0.33622880
Iteration 805, loss = 0.33621596
Iteration 806, loss = 0.33629401
Iteration 807, loss = 0.33588310
Iteration 808, loss = 0.33604859
Iteration 809, loss = 0.33562015
Iteration 810, loss = 0.33612681
Iteration 811, loss = 0.33597556
Iteration 812, loss = 0.33559908
Iteration 813, loss = 0.33587057
Iteration 814, loss = 0.33569055
Iteration 815, loss = 0.33544994
Iteration 816, loss = 0.33554771
Iteration 817, loss = 0.33549889
Iteration 818, loss = 0.33576919
Iteration 819, loss = 0.33495150
Iteration 820, loss = 0.33703042
Iteration 821, loss = 0.33570982
Iteration 822, loss = 0.33590792
Iteration 823, loss = 0.33586422
Iteration 824, loss = 0.33493502
Iteration 

MLPClassifier(max_iter=2000, random_state=21, tol=1e-09, verbose=10)

In [654]:
# Predict on the test set
y_pred = nn.predict(x_train)

In [655]:
print(y_pred.mean())
print(y_train.mean())

0.31425364758698093
0.3838383838383838


In [656]:
final_predictions = pd.DataFrame({'PassengerId': passengerid_test, 'Survived': predictions_rfc})
final_predictions = final_predictions.set_index('PassengerId')
final_predictions

,Survived
PassengerId,
892,0
893,0
894,0
895,1
896,0
...,...
1305,0
1306,1
1307,0


In [657]:
final_predictions.to_csv('survival_submission.csv')